<a href="https://colab.research.google.com/github/madan-genai/NLP_-/blob/main/Fake_News_detection_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

# 1. Importing Libraries

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 2. Importing the Dataset

In [12]:
data = pd.read_csv("/content/news.csv", on_bad_lines='skip', engine='python')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# 3. Preprocessing Dataset

In [14]:
data = data.drop(["Unnamed: 0"],axis=1)
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# 4. Data Encoding

In [16]:
le = preprocessing.LabelEncoder()
le.fit(data["label"])
data['label'] = le.transform(data["label"])

# 5. Variables Setup

In [17]:
embedding_dim = 50
max_length = 54
padding_type = 'post'
trunc_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = 0.1

# 6. Tokenization

In [19]:
title=[]
text=[]
labels=[]
for x in range(training_size):
  title.append(data["title"][x])
  text.append(data["text"][x])
  labels.append(data["label"][x])
tokenizer1=Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1=tokenizer1.word_index
vocab_size1=len(word_index1)
sequences1=tokenizer1.texts_to_sequences(title)
padded1=pad_sequences(sequences1,padding=padding_type, truncating=trunc_type)

# 7. Splitting Data for Training and Testing

In [20]:
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

# 8. Reshaping Data for LSTM

In [21]:
training_sequences1=np.array(training_sequences1)
test_sequences1=np.array(test_sequences1)

# 9. Generating Word Embedding

In [22]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

--2026-02-04 21:32:12--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.84MB/s    in 2m 45s  

2026-02-04 21:34:57 (4.99 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [23]:
embedding_index={}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size1 + 1, embedding_dim))

for word, i in word_index1.items():
    if i < vocab_size1:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# 10. Model Architecture

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1 + 1, embedding_dim, input_length=max_length,
                              weights=[embedding_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [26]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │       377,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 377,600 (1.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 377,600 (1.44 MB)

# 11. Training the Model

In [31]:
history = model.fit(
    training_sequences1,
    np.array(training_labels),
    epochs=50,
    validation_data=(test_sequences1, np.array(test_labels)),
    verbose=2
)

Epoch 1/50
85/85 - 1s - 15ms/step - accuracy: 0.9796 - loss: 0.0587 - val_accuracy: 0.7100 - val_loss: 1.0730
Epoch 2/50
85/85 - 1s - 14ms/step - accuracy: 0.9759 - loss: 0.0662 - val_accuracy: 0.7533 - val_loss: 1.0224
Epoch 3/50
85/85 - 1s - 16ms/step - accuracy: 0.9826 - loss: 0.0508 - val_accuracy: 0.7367 - val_loss: 1.0705
Epoch 4/50
85/85 - 1s - 15ms/step - accuracy: 0.9859 - loss: 0.0424 - val_accuracy: 0.7467 - val_loss: 1.1303
Epoch 5/50
85/85 - 1s - 15ms/step - accuracy: 0.9800 - loss: 0.0563 - val_accuracy: 0.7300 - val_loss: 0.9939
Epoch 6/50
85/85 - 2s - 24ms/step - accuracy: 0.9815 - loss: 0.0490 - val_accuracy: 0.7167 - val_loss: 1.0852
Epoch 7/50
85/85 - 2s - 27ms/step - accuracy: 0.9793 - loss: 0.0562 - val_accuracy: 0.7367 - val_loss: 1.0188
Epoch 8/50
85/85 - 1s - 15ms/step - accuracy: 0.9859 - loss: 0.0430 - val_accuracy: 0.7367 - val_loss: 1.2958
Epoch 9/50
85/85 - 1s - 14ms/step - accuracy: 0.9830 - loss: 0.0458 - val_accuracy: 0.7167 - val_loss: 1.0792
Epoch 10/5

In [32]:
X = "Karry to go to France in gesture of sympathy"

sequences = tokenizer1.texts_to_sequences([X])
sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
if model.predict(sequences, verbose=0)[0][0] >= 0.5:
    print("This news is True")
else:
    print("This news is False")

This news is False
